What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [ ]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames

In [ ]:
# Constants and Parameters
const n, a, b = 2, 4, 12
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref # Objective function

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [3]:
# rand_center = [2*rand()-1, 2*rand()-1]*.75; # Random center
# rand_center = [0.4694712315165298, -0.45637754560934185]
rand_center = [0.0, 0.0];

In [ ]:
d = 26 # Initial Degree 
SMPL = 300 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max = d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

Solve the system of partial derivatives using `Msolve`. For degree `26` on tref centered at `rand_center = [0.4694712315165298, -0.45637754560934185]` and `a/b = 7//12`, `msolve runs single threaded in 5m 2.2 s. With 10 threads, it runs in 4m 48.2 s. 
The slow part of the process is the real root isolation it would seem. 

In [ ]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=true)

In [ ]:
df_lege = solve_and_parse(pol_lege, x, f, TR, basis=:legendre)
sort!(df_lege, :z, rev=true)

In [ ]:
using Optim
df_cheb.steps = zeros(nrow(df_cheb))
df_cheb.converged = zeros(nrow(df_cheb))

for i in 1:nrow(df_cheb)
    # println("Optimizing for point $i")
    x0 = [df_cheb.x1[i], df_cheb.x2[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)

    # df.dist_to_loc_min[i] = distance
    df_cheb.steps[i] = steps
    df_cheb.converged[i] = min_value
    summary(res)
end
df_cheb

In [ ]:
using GLMakie
function plot_polyapprox_rotate(pol::ApproxPoly, TR::test_input, df::DataFrame)
    # Extract coordinates and function values
    coords = pol.scale_factor * pol.grid .+ TR.center'
    z_coords = pol.z

    if size(coords)[2] == 2  # Plot if the dimensions are 2
        fig = Figure(size=(800, 600))
        ax = Axis3(fig[1, 1], title="Trefethen Function",
            xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Initialize the Z grid with NaN
        Z = fill(NaN, (length(y_unique), length(x_unique)))

        # Fill the Z grid by matching coordinates
        for (idx, (x, y, z)) in enumerate(zip(coords[:, 1], coords[:, 2], z_coords))
            i = findlast(≈(y), y_unique)
            j = findlast(≈(x), x_unique)
            if !isnothing(i) && !isnothing(j)
                Z[j, i] = z
            end
        end

        # Create surface plot
        surface!(ax, x_unique, y_unique, Z,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Plot the critical points
        scatter!(ax, df.x1, df.x2,
            df.z,
            markersize=10,
            color=:orange,
            label="Chebyshev approximant critical points")

        # Record rotating animation
        record(fig, "trefethern_rotation.mp4", 1:240; framerate=30) do frame
            ax.azimuth[] = 1.7pi + 0.4 * sin(2pi * frame / 240)
            ax.elevation[] = pi / 4 + 0.3 * cos(2pi * frame / 240)
        end

        display(fig)
        return fig
    end
end

In [52]:
plot_polyapprox_rotate(pol_cheb, TR, df_lege)

In [53]:
# GLMakie.closeall()